In [ ]:
pip install cuda-python torch torchvision torchaudio Cython --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mkdir "/content/drive2/"
%cp -av "/content/drive/MyDrive/data" "/content/drive2/"
%cp -av "/content/drive/MyDrive/garbage_files/" "/content/drive2/"

In [2]:
import os
import torch
import torchvision
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import make_grid
import numpy as np

def load_data(path: str):
    with open(path, "r") as f:
        files = f.readlines()
        targets = []
        images = []

        for line in files:
            name, target = line.split(" ")
            if name.startswith("paper"):
                path = f"/content/drive2/data/paper/{name}"
            elif name.startswith("glass"):
                path = f"/content/drive2/data/glass/{name}"
            elif name.startswith("metal"):
                path = f"/content/drive2/data/metal/{name}"
            elif name.startswith("cardboard"):
                path = f"/content/drive2/data/cardboard/{name}"
            elif name.startswith("plastic"):
                path = f"/content/drive2/data/plastic/{name}"
            elif name.startswith("trash"):
                path = f"/content/drive2/data/trash/{name}"

            targets.append(int(target)-1)
            image = torchvision.io.read_image(path)
            images.append(image)

    return images, targets



class TrashDataset(Dataset):

  def __init__(self, images, targets):
    self.targets = targets
    self.images = images

  def __len__(self):
    return len(self.targets)

  transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor()])

  def __getitem__(self, index):
    return self.targets[index], self.transform(self.images[index])


class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = torchvision.models.vgg19(pretrained=True)
    self.fc1 = nn.Linear(1000, 6)
  def forward(self, x):
    x = self.network(x)
    x = self.fc1(x)
    return x

In [ ]:
import os
import torch
import torchvision
from torch.optim import lr_scheduler
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torchvision.utils import make_grid
import numpy as np


def load_data(path: str):
    with open(path, "r") as f:
        files = f.readlines()
        targets = []
        images = []

        for line in files:
            name, target = line.split(" ")
            if name.startswith("paper"):
                path = f"/content/drive2/data/paper/{name}"
            elif name.startswith("glass"):
                path = f"/content/drive2/data/glass/{name}"
            elif name.startswith("metal"):
                path = f"/content/drive2/data/metal/{name}"
            elif name.startswith("cardboard"):
                path = f"/content/drive2/data/cardboard/{name}"
            elif name.startswith("plastic"):
                path = f"/content/drive2/data/plastic/{name}"
            elif name.startswith("trash"):
                path = f"/content/drive2/data/trash/{name}"

            targets.append(int(target)-1)
            image = torchvision.io.read_image(path)
            images.append(image)

    return images, targets


class TrashDataset(Dataset):

  def __init__(self, images, targets):
    self.targets = targets
    self.images = images

  def __len__(self):
    return len(self.targets)

  transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor()])

  def __getitem__(self, index):
    return self.targets[index], self.transform(self.images[index])


class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.network = torchvision.models.resnet152(pretrained=True)
    self.fc1 = nn.Linear(1000, 6)
  def forward(self, x):
    x = self.network(x)
    x = self.fc1(x)
    return x


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

batch_size = 40
cv_scores = [0]

train_images, train_targets = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_train.txt")
train_dataset = TrashDataset(images=train_images, targets=train_targets)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

cv_images, cv_targets = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_val.txt")
cv_dataset = TrashDataset(images=cv_images, targets=cv_targets)
cv_loader = torch.utils.data.DataLoader(cv_dataset, batch_size=batch_size, shuffle=True, num_workers=0)


net = Net()
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

for epoch in range(1):
    for idx, data in enumerate(train_loader):
        targets, inputs = data

        if torch.cuda.is_available():
          targets, inputs = targets.cuda(), inputs.cuda()

        optimizer_ft.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer_ft.step()
        print(f'[{epoch + 1}, {idx + 1:5d}] loss: {loss}')

        correct = 0
        total = 0
    with torch.no_grad():
        for data in cv_loader:
            targets, inputs = data

            if torch.cuda.is_available():
              targets, inputs = targets.cuda(), inputs.cuda()
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

        cv_score = 100 * correct // total
        print(f'Accuracy of the network on the cv images: {100 * correct // total} %')
        
        if cv_score > max(cv_scores):    
          torch.save(net, f"/content/drive/MyDrive/models/epoch.max")

        cv_scores.append(100 * correct // total)



    
print('Finished Training')
print(cv_scores)

In [64]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

classes = ['Glas', 'Papier', 'Pappe', 'Plastik', 'Metall', 'Müll']
batch_size = 64
net = Net()
net.to(device)

model = torch.load("/content/drive/MyDrive/models/epoch.max")
try_image, try_target = load_data("/content/drive/MyDrive/garbage_files/one-indexed-files-notrash_test.txt")
try_dataset = TrashDataset(images=try_image, targets=try_target)
try_loader = torch.utils.data.DataLoader(try_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

model.eval()
correct = 0
total = 0

types = [[0,0,0,0,0,0],[0,0,0,0,0,0]]

with torch.no_grad():
  for data in try_loader:
    targets, inputs = data
    targets, inputs = targets.cuda(), inputs.cuda()
    # calculate outputs by running images through the network
    outputs = model(inputs)
    # the class with the highest energy is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    for i in range(0, int(predicted.shape[0] - 1)):
      types[0][predicted[i]] = types[0][predicted[i]] + 1
      types[1][predicted[i]] = types[1][predicted[i]] + int(predicted[i] == targets[i])
    total += targets.size(0)
    correct += (predicted == targets).sum().item()
  print(f'Accuracy of the network on all test images: {100 * correct // total} %')
  print(f'Accuracy of the network on glass images: {100 * types[1][0] // types[0][0]} %')
  print(f'Accuracy of the network on paper images: {100 * types[1][1] // types[0][1]} %')
  print(f'Accuracy of the network on cardboard images: {100 * types[1][2] // types[0][2]} %')
  print(f'Accuracy of the network on plastic images: {100 * types[1][3] // types[0][3]} %')
  print(f'Accuracy of the network on metal images: {100 * types[1][4] // types[0][4]} %')
  print(f'Accuracy of the network on trash images: {100 * types[1][5] // types[0][5]} %')

Accuracy of the network on all test images: 95 %
Accuracy of the network on glass images: 95 %
Accuracy of the network on paper images: 95 %
Accuracy of the network on cardboard images: 97 %
Accuracy of the network on plastic images: 95 %
Accuracy of the network on metal images: 95 %
Accuracy of the network on trash images: 89 %


In [65]:
from os import walk
import os
import json
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

out = ["","","","","",""]
classes = ['Glas', 'Papier', 'Pappe', 'Plastik', 'Metall', 'Muell']
batch_size = 64
net = Net()
net.to(device)

model = torch.load("/content/drive/MyDrive/models/epoch.max")
pathToPic = "/content/drive/MyDrive/einzuordnendesFoto/"
testPic = "/content/drive/MyDrive/einzuordnendesFoto/test.jpg"

filename = (pathToPic +("".join(next(walk(pathToPic), (None, None, []))[2])).replace("['", "").replace("']", ""))
os.rename(filename, testPic)

transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((224, 224)),
     transforms.ToTensor()])

try_image = transform(torchvision.io.read_image(testPic))

model.eval()

with torch.no_grad():
  inputs = try_image
  inputs = inputs.cuda()
  inputs = inputs.unsqueeze(0)
  outputs = model(inputs)
  probability = torch.softmax(outputs.data, 1).cpu().numpy()[0]
  maximum = 0
  for u in range(0,6):
    for i in range(0,6):
      if (probability[i] == max(probability)) :
        maximum = i;
        break
    #print("Müll auf dem Bild entspricht zu " + "{0:.10f}".format(float(probability[maximum]*100)) + "% dem Typ " + classes[maximum])
    out[i] = ("Muell auf dem Bild entspricht zu " + "{0:.10f}".format(float(probability[maximum]*100)) + "% dem Typ " + classes[maximum])
    probability[maximum] = 0
  js = {
    "0" : out[0],
    "1" : out[1],
    "2" : out[2],
    "3" : out[3],
    "4" : out[4],
    "5" : out[5]
  }
  js = json.dumps(js)
  file = open("/content/drive/MyDrive/Output.json", "w")
  file.write(js)
  file.close()